In [33]:
from tensorflow.keras import layers, models, losses, Model
import numpy as np
import tensorflow as tf

In [22]:
import shutil
import os
# Paths
#test_augmentation
original_dir = "/home/mahdi/logo_detection_model/autoencoder/detected_logos/"
train_dir = "/home/mahdi/logo_detection_model/autoencoder/train"
val_dir = "/home/mahdi/logo_detection_model/autoencoder/validation"

# Split data into train and validation sets
for class_name in os.listdir(original_dir):
    class_path = os.path.join(original_dir, class_name)
    train_class_path = os.path.join(train_dir, class_name)
    val_class_path = os.path.join(val_dir, class_name)
    
    os.makedirs(train_class_path, exist_ok=True)
    os.makedirs(val_class_path, exist_ok=True)

    images = os.listdir(class_path)
    split_idx = int(len(images) * 0.8)  # 80% for training, 20% for validation

    for img_name in images[:split_idx]:
        shutil.copy(os.path.join(class_path, img_name), train_class_path)
    for img_name in images[split_idx:]:
        shutil.copy(os.path.join(class_path, img_name), val_class_path)

In [46]:
# import os
# import shutil

# base_dir = "/home/mahdi/logo_detection_model/autoencoder/detected_logos/"
# dummy_dir = os.path.join(base_dir, "dummy_class")

# os.makedirs(dummy_dir, exist_ok=True)

# for file_name in os.listdir(base_dir):
#     file_path = os.path.join(base_dir, file_name)
#     if os.path.isfile(file_path):  # Skip directories
#         shutil.move(file_path, dummy_dir)


In [47]:
# train_generator = train_datagen.flow_from_directory(
#     dir,  # Path to training data
#     target_size=(224, 224),  # Resize images
#     batch_size=32,  # Batch size
#     class_mode=None,  # No class labels, input is also output
#     subset="training"  # Training split
# )

# val_generator = val_datagen.flow_from_directory(
#     dir,  # Path to validation data
#     target_size=(224, 224),  # Resize images
#     batch_size=32,  # Batch size
#     class_mode=None,  # No class labels, input is also output
#     subset="validation"  # Validation split
# )


Found 258 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/home/mahdi/logo_detection_model/autoencoder/train"
val_dir = "/home/mahdi/logo_detection_model/autoencoder/validation"
dir = "/home/mahdi/logo_detection_model/autoencoder/detected_logos/"


def safe_preprocess(img):
    try:
        return img  # Pass through valid images
    except Exception as e:
        print("Skipping corrupted image.")
        return np.zeros((224, 224, 3))  # Placeholder for invalid images


# Training Data Generator
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values
    # validation_split=0.2
    preprocessing_function=safe_preprocess
)

# Validation Data Generator
val_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values
    # validation_split=0.2
    preprocessing_function=safe_preprocess
)

# Create Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,  # Path to training data
    target_size=(224, 224),  # Resize images
    # batch_size=4,  # Batch size
    class_mode=None  # No class labels, input is also output
)

val_generator = val_datagen.flow_from_directory(
     val_dir,  # Path to validation data
    target_size=(224, 224),  # Resize images
    # batch_size=4,  # Batch size
    class_mode=None  # No class labels, input is also output
)

print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {val_generator.samples}")

Found 13 images belonging to 1 classes.
Found 7 images belonging to 1 classes.
Training samples: 13
Validation samples: 7


In [18]:
print(f"Train generator samples: {train_generator.samples}")
print(f"Validation generator samples: {val_generator.samples}")


Train generator samples: 13
Validation generator samples: 7


In [19]:
# Try fetching a batch of data
batch = next(iter(train_generator))
print("Train batch shapes:", [x.shape for x in batch])


Train batch shapes: [(224, 224, 3), (224, 224, 3), (224, 224, 3), (224, 224, 3), (224, 224, 3), (224, 224, 3), (224, 224, 3), (224, 224, 3), (224, 224, 3), (224, 224, 3), (224, 224, 3), (224, 224, 3), (224, 224, 3)]


In [20]:
batch = next(iter(val_generator))
print(f"Input batch shape: {batch.shape}")  # Should match (batch_size, 224, 224, 3)


Input batch shape: (7, 224, 224, 3)


/home/mahdi/logo_detection_model/.venv/lib/python3.12/site-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [21]:
len(train_generator)

1

In [22]:
len(val_generator)

1

In [83]:

# # Autoencoder architecture
# def build_autoencoder(input_shape=(224, 224, 3), latent_dim=128):
#     # Encoder
#     input_img = layers.Input(shape=input_shape)
#     x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input_img)
#     x = layers.MaxPooling2D((2, 2), padding="same")(x)
#     x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
#     x = layers.MaxPooling2D((2, 2), padding="same")(x)
#     x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)
#     x = layers.Flatten()(x)
#     latent = layers.Dense(latent_dim, activation="relu")(x)  # Latent space

#     # Decoder
#     x = layers.Dense(28 * 28 * 128, activation="relu")(latent)
#     x = layers.Reshape((28, 28, 128))(x)
#     x = layers.Conv2DTranspose(64, (3, 3), activation="relu", padding="same")(x)
#     x = layers.UpSampling2D((2, 2))(x)
#     x = layers.Conv2DTranspose(32, (3, 3), activation="relu", padding="same")(x)
#     x = layers.UpSampling2D((2, 2))(x)
#     output_img = layers.Conv2DTranspose(3, (3, 3), activation="sigmoid", padding="same")(x)

#     # Autoencoder Model
#     autoencoder = models.Model(input_img, output_img)
#     encoder = models.Model(input_img, latent)  # Separate encoder model for feature extraction

#     return autoencoder, encoder



In [34]:
from tensorflow.keras import layers, models

# Encoder
encoder_input = layers.Input(shape=(224, 224, 3))
x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(encoder_input)
x = layers.MaxPooling2D((2, 2), padding="same")(x)  # Down to (112, 112, 64)
x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2, 2), padding="same")(x)  # Down to (56, 56, 128)
encoder_output = x

# Decoder
x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(encoder_output)
x = layers.UpSampling2D((2, 2))(x)  # Up to (112, 112, 128)
x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
x = layers.UpSampling2D((2, 2))(x)  # Up to (224, 224, 64)
decoder_output = layers.Conv2D(3, (3, 3), activation="sigmoid", padding="same")(x)  # Match input shape

# Autoencoder Model
decoder_output = layers.Conv2D(3, (3, 3), activation="sigmoid", padding="same")(decoder_output)
autoencoder = Model(encoder_input, decoder_output)
autoencoder = models.Model(encoder_input, decoder_output)
autoencoder.compile(optimizer="adam", loss="mse")

autoencoder.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 56, 56, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_6 (UpSampling2D)  │ (None, 112, 112, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 112, 112, 64)   │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_7 (UpSampling2D)  │ (None, 224, 224, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 224, 224, 3)    │         1,731 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 224, 224, 3)    │            84 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 298,839 (1.14 MB)

 Trainable params: 298,839 (1.14 MB)

 Non-trainable params: 0 (0.00 B)

In [114]:
# # Build and compile the autoencoder
# autoencoder, encoder = build_autoencoder()
# autoencoder.compile(optimizer="adam", loss=losses.MeanSquaredError())

In [35]:
print("Autoencoder input shape:", autoencoder.input_shape)
print("Autoencoder output shape:", autoencoder.output_shape)


Autoencoder input shape: (None, 224, 224, 3)
Autoencoder output shape: (None, 224, 224, 3)


In [25]:
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = val_generator.samples // val_generator.batch_size
print(f"Steps per epoch: {steps_per_epoch}, Validation steps: {validation_steps}")


Steps per epoch: 0, Validation steps: 0


In [10]:
# Check the generators
assert train_generator.samples > 0, "Training directory is empty or misconfigured!"
assert val_generator.samples > 0, "Validation directory is empty or misconfigured!"

steps_per_epoch = max(1, train_generator.samples // train_generator.batch_size)
validation_steps = max(1, val_generator.samples // val_generator.batch_size)


In [118]:
! pip install opencv-python



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [119]:
import cv2
import glob
import numpy as np
all_images = []
img_list = glob.glob('/home/mahdi/logo_detection_model/autoencoder/detected_logos/logo_images/*.jpg')


In [120]:
print(img_list)

['/home/mahdi/logo_detection_model/autoencoder/detected_logos/logo_images/3171_2.jpg', '/home/mahdi/logo_detection_model/autoencoder/detected_logos/logo_images/731_2.jpg', '/home/mahdi/logo_detection_model/autoencoder/detected_logos/logo_images/4925_21.jpg', '/home/mahdi/logo_detection_model/autoencoder/detected_logos/logo_images/971_2.jpg', '/home/mahdi/logo_detection_model/autoencoder/detected_logos/logo_images/1007_4.jpg', '/home/mahdi/logo_detection_model/autoencoder/detected_logos/logo_images/1133_1.jpg', '/home/mahdi/logo_detection_model/autoencoder/detected_logos/logo_images/1082_1.jpg', '/home/mahdi/logo_detection_model/autoencoder/detected_logos/logo_images/2533_3.jpg', '/home/mahdi/logo_detection_model/autoencoder/detected_logos/logo_images/3067_9.jpg', '/home/mahdi/logo_detection_model/autoencoder/detected_logos/logo_images/3162_2.jpg', '/home/mahdi/logo_detection_model/autoencoder/detected_logos/logo_images/5138_2.jpg', '/home/mahdi/logo_detection_model/autoencoder/detected

In [121]:
for image in img_list:
    img = cv2.imread(image,0)
    # img = img.astype('float32') / 255.
    img = np.reshape(img,(224,224,3))
    all_images.append(img)
all_images = np.array(all_images)

ValueError: cannot reshape array of size 119400 into shape (224,224,3)

In [122]:
all_images.shape

AttributeError: 'list' object has no attribute 'shape'

In [123]:
# Train the autoencoder
history = autoencoder.fit(
    train_generator,train_generator,
    validation_data= (val_generator,val_generator ),
    epochs=50,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

ValueError: When providing `x` as a PyDataset, `y` should not be passed. Instead, the targets should be included as part of the PyDataset.

In [26]:
len(train_generator)

1

In [36]:
# Debug the data generators
batch = next(iter(train_generator))
print(f"Batch shape: {batch.shape if batch is not None else 'None'}")


Batch shape: (13, 224, 224, 3)


In [37]:
steps_per_epoch = max(1, train_generator.samples // train_generator.batch_size)
validation_steps = max(1, val_generator.samples // val_generator.batch_size)

print(f"Steps per epoch: {steps_per_epoch}")
print(f"Validation steps: {validation_steps}")


Steps per epoch: 1
Validation steps: 1


In [39]:
batch = next(iter(train_generator))
print(f"Train batch shape: {batch.shape}")


Train batch shape: (13, 224, 224, 3)


In [40]:
batch = next(iter(val_generator))
print(f"Validation batch shape: {batch.shape}")


Validation batch shape: (7, 224, 224, 3)


/home/mahdi/logo_detection_model/.venv/lib/python3.12/site-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [41]:
predictions = autoencoder.predict(batch)
print(f"Prediction shape: {predictions.shape}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
Prediction shape: (7, 224, 224, 3)


In [43]:
batch = next(iter(train_generator))
predictions = autoencoder(batch)
print(f"Prediction shape: {predictions.shape}")


Prediction shape: (13, 224, 224, 3)


In [42]:
history = autoencoder.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    steps_per_epoch=max(1, train_generator.samples // train_generator.batch_size),
    validation_steps=max(1, val_generator.samples // val_generator.batch_size),
    verbose=1
)


Epoch 1/50


ValueError: None values not supported.

In [38]:
# Train the autoencoder
history = autoencoder.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    batch_size = 4,
    # steps_per_epoch=len(train_generator),
    # validation_steps=len(val_generator)
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size

)


Epoch 1/50


ValueError: None values not supported.

In [ ]:
# Save the encoder for feature extraction
encoder.save("logo_encoder.h5")


In [ ]:
# Compare similarity using latent features
def calculate_similarity(encoder, input_logo, valid_logos):
    input_features = encoder.predict(input_logo)
    similarities = []
    for valid_logo in valid_logos:
        valid_features = encoder.predict(valid_logo)
        similarity = np.dot(input_features, valid_features.T) / (
            np.linalg.norm(input_features) * np.linalg.norm(valid_features)
        )
        similarities.append(similarity)
    return similarities



In [ ]:
# Example usage
input_logo = ...  # Load input image and preprocess
valid_logos = [...]  # List of preprocessed valid logos
similarities = calculate_similarity(encoder, input_logo, valid_logos)
